In [1]:
#import stuff and get workspace
import os
import azureml.core
from azureml.core import Workspace, Experiment, Run
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.runconfig import DEFAULT_CPU_IMAGE
from azureml.pipeline.core import Pipeline
from azureml.pipeline.steps import PythonScriptStep
from azureml.widgets import RunDetails
from azureml.core import Workspace, Experiment, Datastore
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.widgets import RunDetails
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import Pipeline, PipelineData
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core.schedule import ScheduleRecurrence, Schedule
from azureml.core.dataset import Dataset
from azureml.core.authentication import InteractiveLoginAuthentication


print("SDK version:", azureml.core.VERSION)
#auth = InteractiveLoginAuthentication(force=True, tenant_id="72f988bf-86f1-41af-91ab-2d7cd011db47")
# ws = Workspace.from_config()

SDK version: 1.0.74


In [5]:
ws = Workspace(subscription_id="bbd86e7d-3602-4e6d-baa4-40ae2ad9303c", resource_group="ManyModelsSA", workspace_name="ManyModelsSAv1")
ws.get_details()

{'id': '/subscriptions/bbd86e7d-3602-4e6d-baa4-40ae2ad9303c/resourceGroups/ManyModelsSA/providers/Microsoft.MachineLearningServices/workspaces/ManyModelsSAv1',
 'name': 'ManyModelsSAv1',
 'location': 'centralus',
 'type': 'Microsoft.MachineLearningServices/workspaces',
 'tags': {},
 'sku': 'Enterprise',
 'workspaceid': '77752be6-01b4-4a3e-9d42-03c9c0d6248f',
 'description': '',
 'friendlyName': '',
 'creationTime': '2019-11-18T20:17:22.9729795+00:00',
 'containerRegistry': '/subscriptions/bbd86e7d-3602-4e6d-baa4-40ae2ad9303c/resourceGroups/ManyModelsSA/providers/Microsoft.ContainerRegistry/registries/manymodelssa8faf85d6',
 'keyVault': '/subscriptions/bbd86e7d-3602-4e6d-baa4-40ae2ad9303c/resourcegroups/manymodelssa/providers/microsoft.keyvault/vaults/manymodelssav16798764194',
 'applicationInsights': '/subscriptions/bbd86e7d-3602-4e6d-baa4-40ae2ad9303c/resourcegroups/manymodelssa/providers/microsoft.insights/components/manymodelssav10130926491',
 'identityPrincipalId': '999f751c-ae2e-4

In [ ]:
# !pip install azureml-contrib-pipeline-steps
#!az ml

In [6]:
# ws = Workspace.from_config()

# create a new runconfig object
run_config = RunConfiguration()
run_config.environment.docker.enabled = True
run_config.environment.docker.base_image = DEFAULT_CPU_IMAGE
run_config.environment.python.conda_dependencies = CondaDependencies.create(conda_packages=['scikit-learn'])

#choose a compute target
compute = AmlCompute(ws, "cpu-cluster")
dstore = ws.get_default_datastore()

print(dstore.name, dstore.datastore_type, dstore.account_name, dstore.container_name)

workspaceblobstore AzureBlob manymodelssav16457539585 azureml-blobstore-77752be6-01b4-4a3e-9d42-03c9c0d6248f


In [10]:
workercount=3
nodecount=1
timeout=3000

from azureml.pipeline.core import Pipeline, PipelineData

dataset1 = Dataset.Tabular.from_delimited_files(path = (dstore, '3modelsdata/Store2_dominicks.csv'))
dataset2 = Dataset.Tabular.from_delimited_files(path = (dstore, '3modelsdata/Store5_tropicana.csv'))
dataset3 = Dataset.Tabular.from_delimited_files(path = (dstore, '3modelsdata/Store8_minute.maid.csv'))

output_dir = PipelineData(name="3models", 
                          datastore=dstore, 
                          output_path_on_compute="3models/")

from azureml.core import Experiment
experiment = Experiment(ws, 'automl-ojforecasting')

In [29]:
dataset1.to_pandas_dataframe().head()

,WeekStarting,Store,Brand,Quantity,Advert,Price,Revenue
0,1990-06-14,2,dominicks,10560,1,1.59,16790.40
1,1990-07-26,2,dominicks,8000,0,2.69,21520.00
2,1990-08-02,2,dominicks,6848,1,2.09,14312.32
3,1990-08-09,2,dominicks,2880,0,2.09,6019.20
4,1990-08-23,2,dominicks,1600,0,2.09,3344.00


In [11]:
from azureml.core import Environment
from azureml.core.runconfig import CondaDependencies, DEFAULT_CPU_IMAGE

batch_conda_deps = CondaDependencies.create(pip_packages=['sklearn'])

batch_env = Environment(name="manymodels_environment")
batch_env.python.conda_dependencies = batch_conda_deps
batch_env.docker.enabled = True
batch_env.docker.base_image = DEFAULT_CPU_IMAGE

In [18]:
from azureml.contrib.pipeline.steps import ParallelRunStep, ParallelRunConfig
datasetname='store'

tags1={}
tags1['dataset']=datasetname
tags1['nodes']=nodecount
tags1['workers-per-node']=workercount
tags1['timeout']=timeout

parallel_run_config = ParallelRunConfig(
    source_directory='./scripts',
    entry_script='train.py',
    mini_batch_size="1",
    run_invocation_timeout=timeout,
    error_threshold=10,
    output_action="summary_only",
    environment=batch_env,
    process_count_per_node=workercount,
    compute_target=compute,
    node_count=nodecount)

In [19]:
parallelrun_step = ParallelRunStep(
    name="many-models-training",
    parallel_run_config=parallel_run_config,
    inputs=[dataset1.as_named_input(datasetname), dataset2.as_named_input(datasetname), dataset3.as_named_input(datasetname)],
    output=output_dir,
    models=[],
    arguments=[],
    allow_reuse=False
)

In [20]:

pipeline = Pipeline(workspace=ws, steps=[parallelrun_step])

run1 = experiment.submit(pipeline,tags=tags1)
#RunDetails(run1).show()

Created step many-models-training [be5bdcc5][9695b249-15a3-473e-bcd3-9baa4b0d9054], (This step will run and generate new outputs)
Submitted PipelineRun 6aafdd1d-c357-4aa1-a05c-2b4f448d7e56


WARNING - Converting non-string tag to string: (nodes: 1)
WARNING - Converting non-string tag to string: (workers-per-node: 3)
WARNING - Converting non-string tag to string: (timeout: 3000)


Link to Azure Machine Learning studio: https://ml.azure.com/experiments/automl-ojforecasting/runs/6aafdd1d-c357-4aa1-a05c-2b4f448d7e56?wsid=/subscriptions/bbd86e7d-3602-4e6d-baa4-40ae2ad9303c/resourcegroups/ManyModelsSA/workspaces/ManyModelsSAv1


In [ ]:
run1

In [ ]:
%%writefile ./code/train.py

import pandas as pd
import os
import uuid

from azureml.core.model import Model
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
import pickle
from azureml.core import Experiment, Workspace, Run
from azureml.core import ScriptRunConfig
import datetime
from entry_script_helper import EntryScriptHelper
import logging

from sklearn.externals import joblib
from joblib import dump, load

thisrun = Run.get_context()
#childrun=thisrun

LOG_NAME = "user_log"

def init():
    EntryScriptHelper().config(LOG_NAME)
    logger = logging.getLogger(LOG_NAME)
    output_folder = os.path.join(os.environ.get("AZ_BATCHAI_INPUT_AZUREML", ""), "temp/output")
    logger.info(f"{__file__}.output_folder:{output_folder}")
    logger.info("init()")
    return

def run(input_data):
    logger = logging.getLogger(LOG_NAME)
    os.makedirs('./outputs', exist_ok=True)
    resultList = []
    logger.info('processing all files')
    for file in input_data:
        u1 = uuid.uuid4()
        mname='sklearn_model_'+str(u1)[0:16]
        #for w in range(0,1):
        with thisrun.child_run(name=mname) as childrun:
            for w in range(0,5):
                thisrun.log(mname,str(w))
            date1=datetime.datetime.now()
            logger.info('starting ('+file+') ' + str(date1))
            childrun.log(mname,'starttime-'+str(date1))
            # 1.read in the data file
            data = pd.read_csv(file,header=None)
            logger.info(data.head())
            # 2. set the data up for training
            train,test=train_test_split(data,test_size=0.3)
            train_X=train[train.columns[0:6]]
            train_y=train[train.columns[-1]]
            test_X=test[train.columns[0:6]]
            test_y=train[train.columns[-1]]
            # 3. train a decision tree
            dtmodel=LinearRegression()
            dtmodel.fit(train_X,train_y)
            logger.info('done training')
            # 4. save the model
            logger.info(dtmodel)
            logger.info(mname)
            with open(mname, 'wb') as file:
                joblib.dump(value=dtmodel, filename=os.path.join('./outputs/', mname))
            # 5. Register the model
            ws1 = childrun.experiment.workspace
            try:
                childrun.upload_file(mname, os.path.join('./outputs/', mname))
            except:
                logger.info('dont need to upload')
            logger.info('register model, skip the outputs prefix')
            Model.register(workspace=ws1, model_path=os.path.join('./outputs/', mname), model_name=mname, model_framework='sklearn')
            #you can return anything you want
            date2=datetime.datetime.now()
            logger.info('ending ('+str(file)+') ' + str(date2))

            #log some metrics
            childrun.log(mname,'endtime-'+str(date2))
            childrun.log(mname,'auc-1')
        resultList.append(True)
    return resultList




'''
#cols for IRIS
print(data.head())
train,test=train_test_split(data,test_size=0.3)
train_X=train[[0,1,2,3]]
train_y=train[4]
test_X=test[[0,1,2,3]]
test_y=train[4]
dtmodel=DecisionTreeClassifier()
dtmodel.fit(train_X,train_y)''' 

In [ ]:
run= Run(experiment, run_id='1639679a-3711-4840-bad9-067b0c0352f1')
run.cancel()

In [ ]:
import pandas as pd

from azureml.core.model import Model
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn import metrics
import pickle
from azureml.core import Experiment, Workspace, Run
from azureml.core import ScriptRunConfig
import datetime

data = pd.read_csv('./iris0.txt',header=None)
print(data.head())
print('4')
train,test=train_test_split(data,test_size=0.3)
print('5')
train_X=train[[0,1,2,3]]
print('6')
train_y=train[4]
print('7')
test_X=test[[0,1,2,3]]
test_y=train[4]
print('8')
dtmodel=DecisionTreeClassifier()
print('9')
dtmodel.fit(train_X,train_y)
print('10')
print(dtmodel)
filename = './sklearn_model.pkl'
pickle.dump(dtmodel, open(filename, 'wb'))
model = Model.register(model_name='sklearn_model_e5ebe522',
                   model_path='./sklearn_model.pkl',
                   tags = {'area': "MNIST", 'type': "sklearn"},
                   description = "identify numbers", workspace=ws)

In [ ]:
data = pd.read_csv('/mnt/azmnt/code/Users/buehrer/many_models/code/dota2train.csv',header=None)
print(data.head())
train,test=train_test_split(data,test_size=0.3)
train_X=train[train.columns[1:116]]
train_y=train[[0]]
test_X=test[train.columns[1:116]]
test_y=train[[0]]
dtmodel=DecisionTreeClassifier()
dtmodel.fit(train_X,train_y)

In [ ]:
!pwd